# 1. References

Title: Title Machine Learning, NLP: Text Classification using scikit-learn, python and NLTK.

Link: https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a


# 2. Imports

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import nltk
# nltk.download()
from nltk.stem.snowball import SnowballStemmer

# 3. Data cleaning

In [36]:
file_name = 'TED_Talks_by_ID_plus-transcripts-and-LIWC-and-MFT-plus-views.csv'
# file_name = 'TED_Transcripts_short.csv'
df = pd.read_csv('../owentemple-ted-talks-complete-list/{}'.format(file_name))
df.head()

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,...,harm_vice,fairness_virtue,fairness_vice,ingroup_virtue,ingroup_vice,authority_virtue,authority_vice,purity_virtue,purity_vice,morality_general
0,1,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2,2006,TED2006,0:16:17,...,0.04,0.00,0.00,0.48,0.00,0.22,0.0,0.00,0.00,0.22
1,2,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2,2006,TED2006,0:15:06,...,0.04,0.00,0.00,0.30,0.00,0.11,0.0,0.11,0.04,0.15
2,3,Ashraf Ghani,How to rebuild a broken state,http://www.ted.com/talks/view/id/3,Ashraf Ghani's passionate and powerful 10-minu...,http://www.ted.com/talks/view/id/3/transcript?...,7,2005,TEDGlobal 2005,0:18:45,...,0.12,0.16,0.04,0.32,0.12,0.20,0.0,0.04,0.04,0.08
3,4,Burt Rutan,The real future of space exploration,http://www.ted.com/talks/view/id/4,"In this passionate talk, legendary spacecraft ...",http://www.ted.com/talks/view/id/4/transcript?...,2,2006,TED2006,0:19:37,...,0.19,0.00,0.00,0.19,0.00,0.10,0.0,0.00,0.00,0.16
4,5,Chris Bangle,Great cars are great art,http://www.ted.com/talks/view/id/5,American designer Chris Bangle explains his ph...,http://www.ted.com/talks/view/id/5/transcript?...,2,2002,TED2002,0:20:04,...,0.05,0.03,0.00,0.39,0.00,0.05,0.0,0.00,0.03,0.13


In [37]:
df.iloc[:,:15].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2475 entries, 0 to 2474
Data columns (total 15 columns):
id                      2475 non-null int64
speaker                 2475 non-null object
headline                2475 non-null object
URL                     2475 non-null object
description             2475 non-null object
transcript_URL          2386 non-null object
month_filmed            2475 non-null int64
year_filmed             2475 non-null int64
event                   2475 non-null object
duration                2475 non-null object
date_published          2475 non-null object
views_as_of_06162017    2474 non-null float64
tags                    2475 non-null object
transcript              2386 non-null object
notes                   4 non-null object
dtypes: float64(1), int64(3), object(11)
memory usage: 290.1+ KB


In [38]:
df = df.dropna(subset=['transcript'])
df = df.reset_index(drop=True)
df.iloc[:,:15].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2386 entries, 0 to 2385
Data columns (total 15 columns):
id                      2386 non-null int64
speaker                 2386 non-null object
headline                2386 non-null object
URL                     2386 non-null object
description             2386 non-null object
transcript_URL          2386 non-null object
month_filmed            2386 non-null int64
year_filmed             2386 non-null int64
event                   2386 non-null object
duration                2386 non-null object
date_published          2386 non-null object
views_as_of_06162017    2386 non-null float64
tags                    2386 non-null object
transcript              2386 non-null object
notes                   4 non-null object
dtypes: float64(1), int64(3), object(11)
memory usage: 279.7+ KB


In [39]:
df['transcript'][0]

'0:14\r\r\rThank you so much, Chris.\rAnd it\'s truly a great honor\rto have the opportunity\rto come to this stage twice;\rI\'m extremely grateful.\rI have been blown away by this conference,\rand I want to thank all of you\rfor the many nice comments\rabout what I had to say the other night.\rAnd I say that sincerely,\rpartly because (Mock sob)\rI need that.\r\r\r\r\r 0:40\r\r\r(Laughter)\r\r\r\r\r 0:45\r\r\rPut yourselves in my position.\r\r\r\r\r 0:47\r\r\r(Laughter)\r\r\r\r\r 0:54\r\r\rI flew on Air Force Two for eight years.\r\r\r\r\r 0:57\r\r\r(Laughter)\r\r\r\r\r 0:59\r\r\rNow I have to take off my shoes\ror boots to get on an airplane!\r\r\r\r\r 1:02\r\r\r(Laughter)\r\r\r\r\r 1:05\r\r\r(Applause)\r\r\r\r\r 1:11\r\r\rI\'ll tell you one quick story\rto illustrate what\rthat\'s been like for me.\r\r\r\r\r 1:16\r\r\r(Laughter)\r\r\r\r\r 1:18\r\r\rIt\'s a true story \revery bit of this is true.\r\r\r\r\r 1:21\r\r\rSoon after Tipper and I left the \r(Mock sob) White House \r\r\r\r\r

In [40]:
import string
df['transcript'] = df['transcript'].str.replace(r'\d+','')
df['transcript'] = df['transcript'].str.replace(r'<.*?>','')
for i in string.punctuation:
    if i == "'":
        df['transcript'] = df['transcript'].str.replace(i,'')
    else:
        df['transcript'] = df['transcript'].str.replace(i,' ')
df['transcript'] = df['transcript'].str.replace("'",'')
df['transcript'] = df['transcript'].str.replace('\s+',' ')

In [41]:
df['transcript'][0]

' Thank you so much Chris And its truly a great honor to have the opportunity to come to this stage twice Im extremely grateful I have been blown away by this conference and I want to thank all of you for the many nice comments about what I had to say the other night And I say that sincerely partly because Mock sob I need that Laughter Put yourselves in my position Laughter I flew on Air Force Two for eight years Laughter Now I have to take off my shoes or boots to get on an airplane Laughter Applause Ill tell you one quick story to illustrate what thats been like for me Laughter Its a true story every bit of this is true Soon after Tipper and I left the Mock sob White House Laughter we were driving from our home in Nashville to a little farm we have miles east of Nashville Driving ourselves Laughter I know it sounds like a little thing to you but Laughter I looked in the rear view mirror and all of a sudden it just hit me There was no motorcade back there Laughter Youve heard of phant

## 3.1 Compare Tags to get a single tag

In [42]:
def tag_selection(df=df):
    complete_transcripts_tags = []
    for rows, value in df.iterrows():
        indiv_tags = value['tags'].split(',')
        longest_tag = max(indiv_tags, key=len)
        indiv_transcript_tags = [value['transcript'], longest_tag]
        complete_transcripts_tags.append(indiv_transcript_tags)
    return pd.DataFrame(complete_transcripts_tags, columns=['transcript', 'tags'])
tag_cleaned = tag_selection()
tag_cleaned

,transcript,tags
0,Thank you so much Chris And its truly a great...,alternative energy
1,In terms of invention Id like to tell you the...,alternative energy
2,A public Dewey long ago observed is constitut...,global development
3,I want to start off by saying Houston we have...,industrial design
4,What I want to talk about is as background is...,industrial design
5,At the break I was asked by several people ab...,entrepreneur
6,Music The Sound of Silence Simon Garfunkel He...,interface design
7,Kurt Andersen Like many architects David is a...,disaster relief
8,As you pointed out every time you come here y...,industrial design
9,With all the legitimate concerns about AIDS a...,global issues


In [50]:
# Check unique tags
tags_cleaned_up = tag_cleaned['tags'].unique()
print(len(tags_cleaned_up))

202


# 4. ML part

In [24]:
from sklearn.model_selection import train_test_split

In [43]:
# do the train test split 
transcript = tag_cleaned['transcript'].to_numpy()
tags = tag_cleaned['tags'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
     transcript, tags, test_size=0.2, random_state=42)

In [44]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(1908, 52376)

In [45]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(1908, 52376)

In [46]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)


In [47]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(X_train, y_train)

In [48]:
predicted = text_clf.predict(X_test)
# print(y_test)
# print(predicted)
np.mean(predicted == y_test)

0.12552301255230125